In [1]:
%matplotlib inline

In [2]:
import model_utils
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [3]:
data_dir = 'data/masked_images/data/trainval/breeds_dog/'
test_data_dir = 'data/masked_images/data/test/breeds_dog/'
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
train_transforms = transforms.Compose([transforms.Resize(224),
                                        transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       normalize
                                       ])
test_transforms = transforms.Compose([transforms.Resize(224),
                                        transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       normalize
                                       ])
trainloader, valloader, classes, labels = model_utils.load_split_train_val(data_dir, train_transforms,test_transforms, normalize,.25)

In [4]:
use_gpu = torch.cuda.is_available()

model_dog = torchvision.models.resnet50(pretrained=True)
num_ftrs = model_dog.fc.in_features

model_dog.fc = nn.Linear(num_ftrs, len(classes))

if use_gpu:
    model_dog = model_dog.cuda()

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_dog.parameters(), lr=0.001)

scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

In [5]:
model_dog = model_utils.train_model(model=model_dog,
                           criterion=criterion,
                           optimizer=optimizer_ft,
                           scheduler=scheduler,
                           trainloader=trainloader,
                           valloader=valloader,
                           num_epochs=10)

Epoch 0/9
--------------------
train Loss: 3.3058 Acc: 0.0434
val Loss: 2.9690 Acc: 0.1974
--------------------
Epoch 1/9
--------------------
train Loss: 3.2357 Acc: 0.0548
val Loss: 2.5664 Acc: 0.2916
--------------------
Epoch 2/9
--------------------
train Loss: 2.9978 Acc: 0.1255
val Loss: 1.8653 Acc: 0.5351
--------------------
Epoch 3/9
--------------------
train Loss: 2.1447 Acc: 0.4464
val Loss: 1.5012 Acc: 0.5942
--------------------
Epoch 4/9
--------------------
train Loss: 1.2590 Acc: 0.7078
val Loss: 1.2377 Acc: 0.6443
--------------------
Epoch 5/9
--------------------
train Loss: 0.7537 Acc: 0.8865
val Loss: 1.2092 Acc: 0.7124
--------------------
Epoch 6/9
--------------------
train Loss: 0.6554 Acc: 0.9249
val Loss: 1.1365 Acc: 0.7786
--------------------
Epoch 7/9
--------------------
train Loss: 0.5939 Acc: 0.9329
val Loss: 1.2045 Acc: 0.7154
--------------------
Epoch 8/9
--------------------
train Loss: 0.5406 Acc: 0.9442
val Loss: 1.1867 Acc: 0.7285
-------------

In [6]:
saving_path = './trained_model/resnet50_breed_dog.pth'
torch.save(model_dog, saving_path)

In [7]:
model_utils.print_test_acc(model_dog, test_transforms, normalize, test_data_dir)

Overall accuracy for all breeds 78% 

Accuracy of american_bulldog : 67 %
Accuracy of american_pit_bull_terrier : 17 %
Accuracy of basset_hound : 70 %
Accuracy of beagle : 47 %
Accuracy of boxer : 76 %
Accuracy of chihuahua : 82 %
Accuracy of english_cocker_spaniel : 70 %
Accuracy of english_setter : 92 %
Accuracy of german_shorthaired : 82 %
Accuracy of great_pyrenees : 90 %
Accuracy of havanese : 62 %
Accuracy of japanese_chin : 87 %
Accuracy of keeshond : 79 %
Accuracy of leonberger : 92 %
Accuracy of miniature_pinscher : 57 %
Accuracy of newfoundland : 90 %
Accuracy of pomeranian : 95 %
Accuracy of   pug : 94 %
Accuracy of saint_bernard : 90 %
Accuracy of samoyed : 95 %
Accuracy of scottish_terrier : 94 %
Accuracy of shiba_inu : 92 %
Accuracy of staffordshire_bull_terrier : 62 %
Accuracy of wheaten_terrier : 80 %
Accuracy of yorkshire_terrier : 85 %
